[![Github](https://img.shields.io/github/stars/labmlai/annotated_deep_learning_paper_implementations?style=social)](https://github.com/labmlai/annotated_deep_learning_paper_implementations)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/labmlai/annotated_deep_learning_paper_implementations/blob/master/labml_nn/transformers/basic/autoregressive_experiment.ipynb)

## Transformer Experiment

This trains a simple transformer with
[multi headed attention](https://nn.labml.ai/transformers/mha.html)
introduced in [Attention Is All You Need](https://arxiv.org/abs/1706.03762)
on an NLP auto-regression task (with Tiny Shakespeare dataset).

### Install the packages

In [14]:
!pip install labml labml-nn --upgrade --quiet

### Imports

In [2]:
from labml import experiment
from labml_nn.transformers.basic.autoregressive_experiment import Configs
from labml import experiment
from labml_nn.transformers.basic.autoregressive_experiment import Configs

### Create an experiment

In [3]:
experiment.create(name="transformer", writers={'screen'})


### Configurations

In [4]:

conf = Configs()

Set experiment configurations and assign a configurations dictionary to override configurations

In [8]:
experiment.configs(conf, {#character level tokenizer
                          'tokenizer':'character',
                          #prompt separator is blank
                          'prompt_separator':'',
                          #starting prompt for smapling
                          'prompt':'It is ',
                          # tiny desk dataset
                          'text':'tiny_shakespeare',
                          #context size of %256%
                          'seq_len':512,
                          #train for 32 epochs
                          'epochs': 32,
                          #batch size 32%
                          'batch_size':16,
                          #swithc between training and validation 10 times
                          'inner_iterations':10,

                          #model size
                          'd_model': 256,
                          'transformer.n_heads': 16,
                          'transformer.ffn.d_ff': 1024,
                          'optimizer.optimizer': 'Noam',
                          'optimizer.learning_rate': 1
                          })




Set PyTorch models for loading and saving

### Start the experiment and run the training loop.

In [22]:
import os
import labml.tracker # Import tracker to patch it if needed
from labml.internal.configs.base import Configs as BaseConfigs # Import base Configs class for patching

# Define the path to the problematic .pid file from the error message
pid_file_path = "/content/logs/transformers/8d0f1744c8f711f08a7a0242ac1c000c/run.pid"

# Check if the file exists and delete it
if os.path.exists(pid_file_path):
    os.remove(pid_file_path)
    print(f"Removed stale PID file: {pid_file_path}")

# Patch labml.tracker.set_text if it doesn't exist
if not hasattr(labml.tracker, 'set_text'):
    print("Patching labml.tracker.set_text as it does not exist.")
    labml.tracker.set_text = lambda *args, **kwargs: None

# Patch labml.tracker.debug if it doesn't exist
if not hasattr(labml.tracker, 'debug'):
    print("Patching labml.tracker.debug as it does not exist.")
    labml.tracker.debug = lambda *args, **kwargs: None

# Store the original __setattr__ method of the base Configs class
_original_base_configs_setattr = BaseConfigs.__setattr__

# Define a custom __setattr__ to bypass the 'Cannot set after it was accessed' error for 'state_modules'
def _patched_base_configs_setattr(self, key, value):
    if key == 'state_modules' and hasattr(self, '__cached'):
        # If 'state_modules' is being set, bypass the cache check.
        # Directly set the value in __values and ensure it's not in __cached.
        # This assumes 'state_modules' is a configuration item, stored in __values.
        self.__values[key] = value
        if key in self.__cached:
            del self.__cached[key] # Ensure it's not cached after setting
    else:
        # For all other attributes, or if 'state_modules' logic doesn't apply, use the original method
        _original_base_configs_setattr(self, key, value)

# Apply the patch to the BaseConfigs class
BaseConfigs.__setattr__ = _patched_base_configs_setattr

try:
    # Start the experiment
    with experiment.start():
         conf.run()
finally:
    # Ensure the original __setattr__ is restored after the experiment runs or fails
    BaseConfigs.__setattr__ = _original_base_configs_setattr


Removed stale PID file: /content/logs/transformers/8d0f1744c8f711f08a7a0242ac1c000c/run.pid


ValueError: Cannot set Configs:state_modules after it was accessed